#MIP Merge

This script merges the different data sets (MUP ownership data, MUP MIP panel on the owners and the companies), turns them into panels and cleans the different columns

In [1]:
import pandas as pd
import numpy as np

Load the data into DataFrames

In [125]:
df_ownership = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPOwn.csv", 
                           encoding="ISO-8859-1")
df_companies = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owned.csv", 
                           encoding="ISO-8859-1")
df_owners = pd.read_csv(r"C:\Users\lucas\OneDrive\BA\Data\Ownership_Change\MUPMIP_panel_owner.csv", 
                        encoding="ISO-8859-1")

C:\Users\lucas\AppData\Local\Temp\ipykernel_34780\1245519110.py:3: DtypeWarning: Columns (3,13,32,33,34,35,36,37,39,47,58,59,60,61,62,63,64,65,68,69,72,73,77,82,85,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,183,184,190,192,193,194,195,197,201,202,203,204,215,219,220,221,222,223,224,231,232,235,236,238,239,240,241,242,244,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,370,384,392,393,394,395,396,397,411,413,419,422,423,424,425,426,427,428,4

The flag `b_is_main_owner` is used to seperate minority from majority shareholders (defined as the owner of at least 50% of the equity), and, where there is no information on the percentage owned, only owners with following "characteristics" (dt. Eigenschaft) were considered majority: "Owner" (Inhaber), "Shareholder" (Gesellschafter), "Limited Partner" (Kommanditist), "General Partner" (Komplementär), and "Majority Shareholder" (Hauptaktionär)

In [126]:
df_ownership["b_is_main_owner"] = np.where(df_ownership["b_anteil"] >= 50, True, 
                                           np.where((df_ownership["b_anteil"].isna())
                                           & (df_ownership["b_eigenschaft"].str.contains("Inhaber|Gesellschafter|Kommanditist|Komplementär|Hauptaktionär", regex=True)),
                                             True, False))

Specify the start and end year of the participation, as a step to turn the ownership data into a panel. The entries which have no start nor end date will be assigned all the years where there are observations in the MIP data set (first year 1993, last year 2021). The end date is set to 2023 for all participations which didn't end in the observation period or where there is no information so that 2021 is within the start to end range

In [127]:
df_ownership["b_start_year"] = df_ownership["b_beginn"].astype(str).str[:4]
df_ownership["b_end_year"] = df_ownership["b_ende"].astype(str).str[:4]
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "0.0", 1993, df_ownership["b_start_year"])
df_ownership["b_start_year"] = np.where(df_ownership["b_start_year"] == "nan", 1993, df_ownership["b_start_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "0.0", 2023, df_ownership["b_end_year"])
df_ownership["b_end_year"] = np.where(df_ownership["b_end_year"] == "nan", 2023, df_ownership["b_end_year"])

Parse `b_start_year` and `b_end_year` to integers

In [128]:
df_ownership["b_start_year"] = pd.to_numeric(df_ownership["b_start_year"], downcast="integer")
df_ownership["b_end_year"] = pd.to_numeric(df_ownership["b_end_year"], downcast="integer")

The array `survey_years` contains all the years with sample data for the companies in the MIP panel. Now I will create dummy variables for all sample years, so that we transform the ownership data frame later to a panel

In [129]:
survey_years = np.unique(df_companies["jahr"])
for i in range(len(survey_years)):
    df_ownership[str(survey_years[i])] = np.where((df_ownership["b_start_year"] <= survey_years[i]) 
                                                  & (df_ownership["b_end_year"] > survey_years[i]),True, False)

`df_panel_ownership` now has the structure of a panel data set, after using `df.melt()`

In [130]:
df_panel_ownership = df_ownership.melt(id_vars=['crefo', 'b_crefo', 'b_eigenschaft', 'b_betrag', 'b_anteil', 
                                                'b_beginn','b_ende', 'b_firma', 'b_person', 'welle', 'companyid', 
                                                'ownerid','b_is_main_owner', 'b_start_year', 'b_end_year'],
                                                  var_name="panel_year")

In this step, I filtered data set to only have the years where there was an observation on the owner and to only have the main owners included, and drop the variables afterwards. I also removed the owners which do not have a companyid.

In [131]:
df_panel_ownership = df_panel_ownership[df_panel_ownership["value"] == True]
df_panel_ownership = df_panel_ownership[df_panel_ownership["b_is_main_owner"] == True]
df_panel_ownership = df_panel_ownership[df_panel_ownership["companyid"] >= 0]
df_panel_ownership.drop(labels=["value", "b_is_main_owner"], axis=1, inplace=True)

Currently, we have 48.889 different owners corresponding to 23.146 companies, and 1.1 million observations in our data set

In [132]:
print(len(df_panel_ownership["b_crefo"].unique()))
print(len(df_panel_ownership))
print(len(df_panel_ownership["companyid"].unique()))

48899
1142976
23146


If there are with the same `b_crefo` for the same year, keep the entry with the most recent wave

In [137]:
df_panel_ownership.sort_values(by="welle", axis=0, ascending=False, inplace=True)
df_panel_ownership.drop_duplicates(["b_crefo", "panel_year"], keep="first", inplace=True)

In [ ]:
#Drop all companies where there are more than 5 owners

Inner merge with companies data: the option `inner` when merging the ownership panel data and the companies panel data ensures that only companies with ownership data (and vice-versa, i.e. only ownership data linked to a company) end up in the merged data set

In [138]:
df_panel_ownership["panel_year"] = pd.to_numeric(df_panel_ownership["panel_year"])
df_merged_companies = pd.merge(df_panel_ownership, df_companies, how="inner", left_on=["panel_year", "companyid"], right_on=["jahr", "companyid"])
df_merged_companies

,crefo,b_crefo,b_eigenschaft,b_betrag,b_anteil,b_beginn,b_ende,b_firma,b_person,welle_x,...,ghe3,ghp1,ghp2,ghvarp,maein1,maein2,maein3,maein4,maein5,_merge
0,4070054625,4070293685,Kommanditist,1105000.0,NaN,20080101.0,0.0,nat. Person,nat. Person,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
1,4070054625,4070054622,Kommanditist,195000.0,NaN,0.0,0.0,nat. Person,nat. Person,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
2,4070054625,4070054618,Komplementär,NaN,NaN,19730125.0,0.0,Unternehmen,Unternehmen,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
3,4070054625,4070054623,Kommanditist,130000.0,NaN,0.0,0.0,nat. Person,nat. Person,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
4,4070054625,4070346252,Kommanditist,1170000.0,NaN,20080101.0,0.0,nat. Person,nat. Person,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131825,7330295814,7330295816,Gesellschafter,100000.0,50.0,19900907.0,0.0,nat. Person,nat. Person,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
131826,4090134731,4090172384,Gesellschafter,12500.0,50.0,0.0,0.0,nat. Person,nat. Person,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
131827,4090134731,4090134732,Gesellschafter,12500.0,50.0,0.0,0.0,nat. Person,nat. Person,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)
131828,8110038542,8110038543,Gesellschafter,26000.0,100.0,0.0,0.0,nat. Person,nat. Person,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,matched (3)


21.975 companies contained in the merged data set

In [139]:
len(df_merged_companies["companyid"].value_counts())

21975

Create a file with all variables and their descriptions, in order to sort out which ones are needed

In [13]:
df_columns = pd.DataFrame(df_merged_companies.columns)
df_columns.columns = ["labels"]
df_descriptions_own = pd.read_excel(r"C:\Users\lucas\OneDrive\BA\Data\MUPOwn_panel_variables.xlsx")
df_descriptions_mip = pd.read_excel(r"C:\Users\lucas\OneDrive\BA\Data\MUPMIP_panel_variables.xlsx")
df_columns = pd.merge(df_columns, df_descriptions_own, how="left", left_on="labels", right_on="name")
df_columns = pd.merge(df_columns, df_descriptions_mip, how="left", left_on="labels", right_on="name")
df_columns.to_csv(r"C:\Users\lucas\OneDrive\BA\Data\merged_variables.csv")